In [1]:
from nbimport_helper import nbimport

ALD_lane = nbimport("ALD_Updated_Lane_search")
main = ALD_lane.main

In [ ]:

import cv2
import matplotlib.pyplot as plt
import numpy as np
# import nbimporter
# from ALD_Updated_Lane_search import main


def find_lane_pixels(binary_warped):
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    midpoint = int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])  #have index of max value
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint #have index of max value 
    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))  #this is created as three channel b/w image  to print colorful lanes 
   
    nwindows = 9  #number of sliding windows we wan on whole image 
    margin = 100
    minpix = 50
    window_height = int(binary_warped.shape[0]//9)
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0]) #y indexes of all non zero pixels eg 0,0,0, 0,  0,  0, 1, 1, 1, 2,  2, 3, 3,  3,  3, max till 720 
    nonzerox = np.array(nonzero[1]) #x indexes of all non zero pixels eg 1,3,5,234,256,761,34,55,66,77,88, 55,6,652,657 max till 1200
      
    left_lane_inds = []
    right_lane_inds = []
    
    leftx_current = leftx_base #current starting index of lane
    rightx_current = rightx_base  #current starting index of lane

    for window in range(nwindows):
       
        win_y_low = binary_warped.shape[0] - (window+1)*window_height #0~720
        win_y_high = binary_warped.shape[0] - window*window_height   #0~720
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization ima
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds) # these are indexes for targeted values in nonzerox nonozeroy for right and left 
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Extract left and right line pixel positions
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
    # Avoids an error if the above is not implemented fully
        pass

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx= nonzerox[right_lane_inds]
    righty= nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_warped, left_fit2, right_fit2,boolval):

    # # Find our lane pixels first
    # forfirstframe=boolval
    # if not forfirstframe:
    #     # For all frames after the first, use your tracking/main function.
    #     resultimage, Bleft_fit, Bright_fit,fleft_fitmm, fright_fitmm = main(binary_warped, left_fit2, right_fit2)
        
    #     return resultimage, Bleft_fit, Bright_fit, forfirstframe, fleft_fitmm, fright_fitmm


        
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    
    
   # Fit a second order polynomial to each using `np.polyfit`
    left_fitmn = np.polyfit((lefty*0.0417), (leftx*0.00529), 2)
    right_fitmn = np.polyfit((righty*0.0417), (rightx*0.00529), 2)
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)


    # Generate x and y values for plotting  
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*(ploty**2) + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*(ploty**2) + right_fit[1]*ploty + right_fit[2]
        # print('The function to fit a line!')
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    plt.gca().invert_yaxis()
    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, binary_warped.shape[1]-1)   # width in pixels
    plt.ylim(binary_warped.shape[0]-1, 0) 
    forfirstframe=False
    return out_img, left_fit, right_fit, forfirstframe, left_fitmn, right_fitmn
